In [1]:
import nltk
import pandas
import datetime
import math
from datetime import datetime
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
import time
import string
from nltk.tokenize import RegexpTokenizer
from nltk import WordNetLemmatizer
from nltk.lm import Vocabulary
from nltk.corpus import wordnet
wnLemmatizer=WordNetLemmatizer()
from math import log10
import math
from sklearn.metrics import accuracy_score


model_file="model-2018.txt"
result_file="baseline-result.txt"

## Experiment 2 : Stop Words ##################################################################################
stopwords_filter=False

## Experiment 3 : Word Length Filtering #######################################################################
wordlength_filter=False

## Experiment 4 : Infrequent Word Filtering  ##################################################################
infreqword_filter_a=False
infreqword_filter_b=False

## Experiment 5 : Smoothing  ##################################################################################
delta=0.5


[nltk_data] Downloading package wordnet to /home/nishant/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/nishant/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
## to change the pos tags to the wordnet tags

def get_taging_value(tagging):
    if tagging.upper().startswith("N"):
        return wordnet.NOUN
    elif tagging.upper().startswith("V"):
        return wordnet.VERB
    elif tagging.upper().startswith("J"):
        return wordnet.ADJ
    elif tagging.upper().startswith("R"):
        return wordnet.ADV
    else: 
        return wordnet.NOUN;

In [3]:
## Experiment 2: Stop word
stopwords_file = open('stopWords.txt', 'r',encoding="utf8")
stopwords = stopwords_file.readlines()
stopwords_file.close()
for word_loop in range(0,len(stopwords)):
        stopwords[word_loop]=stopwords[word_loop].strip()
if stopwords_filter:
    model_file="stopword-model.txt"
    result_file="stopword-result.txt"
else:
    stopwords=[]
  
(stopwords)

[]

In [4]:
## Experiment 3 : Word Length Filtering 
if wordlength_filter:
    model_file="wordlength-model.txt"
    result_file="wordlength-result.txt"

word_length_words=set()
def word_length_filtering(word_inp):
    bool_result=len(word_inp)>2  and len(word_inp)<9
    if not bool_result:
        word_length_words.add(word_inp)
    if wordlength_filter:
        return bool_result
    else:
        return True

In [5]:
##check is start and end is characaters and no digits in between
def check_word_correct(word_inp):
    word_inp=word_inp.lower()
    bool_word_filter=True
    if wordlength_filter:
        bool_word_filter=word_length_filtering(word_inp)
    return (not any(ch.isdigit() for ch in word_inp)) and (not word_inp in stopwords) 

In [6]:
##initial values
start_time=time.process_time()
start_time=time.time()
regExpTokenizer=RegexpTokenizer(r'\w+')
vocab_per_post_type=dict()
vocab_per_post_type['story']=dict()
vocab_per_post_type['ask_hn']=dict()
vocab_per_post_type['show_hn']=dict()
vocab_per_post_type['poll']=dict()
post_type_len=dict()
post_type_len['story']=0
post_type_len['ask_hn']=0
post_type_len['show_hn']=0
post_type_len['poll']=0

##vocabulary
vocabulary=set()
vocab_with_freq=dict()
##remove words
remove_words=set()

##number of training records
train_records=0

##list of test data
test_data=[]

input_pandas = pandas.read_csv("hn2018_2019.csv", encoding="utf8")
input_pandas['Created At'] = pandas.to_datetime(input_pandas['Created At'])
input_pandas['Created At']= input_pandas['Created At'].dt.year

In [7]:
from nltk import bigrams


for index,pandas_row in input_pandas.iterrows():
    loop_row=dict(pandas_row)
    year=loop_row["Created At"]
    
    
    ## tokenization
    token_words=regExpTokenizer.tokenize(loop_row["Title"])
    temp_words=nltk.pos_tag(token_words)
    ## lemmatization
    train_words=[wnLemmatizer.lemmatize(w[0], get_taging_value(w[1])) for w in temp_words]
    
    if year==2018:
        line_vocab=list()
        line_remove=set()
        ##bigram
        bigrams=list(nltk.bigrams(token_words))
        bigram_counter=0
        last_added=False
        for bigram in bigrams:
            word1,tag1=temp_words[bigram_counter]
            word2,tag2=temp_words[bigram_counter+1]
            lemma1=train_words[bigram_counter]
            lemma2=train_words[bigram_counter+1]
            if tag1.startswith("N")  and  tag2.startswith("N"):
                if check_word_correct(lemma1) and check_word_correct(lemma2) and check_word_correct(word1) and check_word_correct(word2):
                    bi=lemma1+" "+lemma2
                    line_vocab.append(bi)
                    last_added=True
                elif check_word_correct(lemma1) and check_word_correct(word1):
                    line_vocab.append(lemma1)
                    last_added=False
                else:
                    line_remove.add(lemma1)
                    last_added=False
                
                if bigram_counter==len(train_words)-2:
                    if check_word_correct(lemma2) and check_word_correct(word2):
                        line_vocab.append(lemma2)
                    else:
                        line_remove.add(lemma2)
            else:
                ##unigram
                if not last_added:
                    if check_word_correct(lemma1) and check_word_correct(word1):
                        line_vocab.append(lemma1)
                    else:
                        line_remove.add(lemma1)
                last_added=False
                if bigram_counter==len(train_words)-2:
                    if check_word_correct(lemma2) and check_word_correct(word2):
                        line_vocab.append(lemma2)
                    else:
                        line_remove.add(lemma2)
            bigram_counter=bigram_counter+1  
        
        for rw in line_remove:
            rw=rw.lower()
            remove_words.add(rw)
            
        for voc in line_vocab:
            voc=voc.lower()
            if word_length_filtering(voc):
                vocabulary.add(voc)
                 
        
            ####### for Experiment 4 ##############################################
            if voc not in vocab_with_freq:
                vocab_with_freq[voc]=1
            else:
                vocab_with_freq[voc]=vocab_with_freq[voc]+1
            #######################################################################
            if loop_row["Post Type"] in vocab_per_post_type:
                if voc in vocab_per_post_type[loop_row["Post Type"]]:
                    vocab_per_post_type[loop_row["Post Type"]][voc]=vocab_per_post_type[loop_row["Post Type"]][voc]+1
                else:
                    vocab_per_post_type[loop_row["Post Type"]][voc]=1
            else:
                vocab_per_post_type[loop_row["Post Type"]]=dict()
                vocab_per_post_type[loop_row["Post Type"]][voc]=1
        
        
        if loop_row["Post Type"] in post_type_len:
            post_type_len[loop_row["Post Type"]]=post_type_len[loop_row["Post Type"]]+1
        else:
            post_type_len[loop_row["Post Type"]]=1
        train_records=train_records+1
    elif year==2019:
        test_data.append(loop_row)
print(post_type_len)
print(train_records)
print(time.time()-start_time)

if stopwords_filter:
    for stpwrd in stopwords:
        if stpwrd in remove_words:
            remove_words.remove(stpwrd)
if wordlength_filter:
    for wrdftr in word_length_words:
        #print(wrdftr)
        if wrdftr in remove_words:
            remove_words.remove(wrdftr)
(len(vocabulary))

{'story': 254222, 'ask_hn': 12509, 'show_hn': 10225, 'poll': 25}
276981
361.01934146881104


368819

In [8]:
############ Experiment 4 Part a ##########################

if infreqword_filter_a:
    vocab_freq_sorted = sorted(vocab_with_freq.items(), key=lambda key_value: key_value[1])
    emp4a_remove=set()
    min_freq=20
    for word, freq in vocab_freq_sorted:
         if freq<=min_freq:
            emp4a_remove.add(word)

    for key in vocab_per_post_type.keys():
        for word in emp4a_remove:
            if word in vocab_per_post_type[key]:
                vocab_per_post_type[key][word]=0
            
    for word in emp4a_remove:
        vocabulary.remove(word)
    print(len(vocabulary))

In [9]:
############ Experiment 4 Part b ##########################


if infreqword_filter_b:
    vocab_freq_reverse = sorted(vocab_with_freq.items(), key=lambda key_value: key_value[1], reverse=True)
    percentage=5
    counter_max=math.floor((percentage*len(vocabulary))/100)

    emp4b_remove=set()
    counter_emp4=1

    for word, freq in vocab_freq_reverse:
        if counter_emp4<=counter_max:
            emp4b_remove.add(word)
        counter_emp4=counter_emp4+1

    for key in vocab_per_post_type.keys():
        for word in emp4b_remove:
            if word in vocab_per_post_type[key]:
                vocab_per_post_type[key][word]=0
            
    for key in emp4b_remove:
        vocabulary.remove(key)
    print(len(vocabulary))

In [10]:
## storing the model data in model-2018.txt
vocabulary_len=len(vocabulary)
vocab_sizes=dict()
vocab_sizes['story']=0
vocab_sizes['ask_hn']=0
vocab_sizes['show_hn']=0
vocab_sizes['poll']=0

for key in vocab_per_post_type.keys():
    vocab_sizes[key]=0
    for word_count in vocab_per_post_type[key]:
        vocab_sizes[key]=vocab_sizes[key]+vocab_per_post_type[key][word_count]
    print(key,vocab_sizes[key])
    
vocabulary=sorted(vocabulary)
prob_word=[]
for vocab_word in vocabulary:
    word_loop=dict()
    word_loop['word']=vocab_word
    for key in vocab_per_post_type.keys():
        if not vocab_word in vocab_per_post_type[key]:
            vocab_per_post_type[key][vocab_word]=0
        word_probability=(vocab_per_post_type[key][vocab_word]+delta)/(vocab_sizes[key]+(vocabulary_len))
        ## frequency,probability
        word_loop[key]=(vocab_per_post_type[key][vocab_word],log10(word_probability))
    prob_word.append(word_loop)
print(vocabulary_len)

story 1879011
ask_hn 115744
show_hn 86845
poll 197
368819


In [11]:
## saving vocabulary file
vocabulary_file= open("vocabulary.txt","w+", encoding="utf-8")
for vocab_word in vocabulary:
    vocabulary_file.write("%s\n"%(vocab_word))
vocabulary_file.close()    

## saving remove word file
removeword_file= open("remove_word.txt","w+", encoding="utf-8")
for removeword in remove_words:
    removeword_file.write("%s\n"%(removeword))
removeword_file.close() 

## saving model-2018
model_train_file= open(model_file,"w+", encoding="utf-8")
counter=1
for loop in prob_word:
    model_train_file.write("%d  %s  %d  %f  %d  %f  %d  %f  %d  %f\n" %(counter, loop['word'], loop['story'][0], loop['story'][1], loop['ask_hn'][0], loop['ask_hn'][1], loop['show_hn'][0], loop['show_hn'][1], loop['poll'][0], loop['poll'][1]))
    counter=counter+1
model_train_file.close()

In [12]:
vocab_dict=dict.fromkeys(vocabulary,0)

In [13]:

test_count=0
test_right=0
start_time=time.time()
output_given=list()
output_pred=list()
output_lines=list()
counter=1
for loop_row in test_data:
    output_one_line=dict()
    output_one_line['counter']=counter
    output_one_line['title']=loop_row["Title"]
    output_one_line['trueval']=loop_row["Post Type"]
    token_words=regExpTokenizer.tokenize(loop_row["Title"])
    output_given.append(loop_row["Post Type"])
    temp_words=nltk.pos_tag(token_words)
    train_words=[wnLemmatizer.lemmatize(w[0], get_taging_value(w[1])) for w in temp_words]
    line_vocab=list()    
    ##bigram
    bigrams=list(nltk.bigrams(token_words))
    bigram_counter=0
    last_added=False
    for bigram in bigrams:
        word1,tag1=temp_words[bigram_counter]
        word2,tag2=temp_words[bigram_counter+1]
        lemma1=train_words[bigram_counter]
        lemma2=train_words[bigram_counter+1]
        if tag1.startswith("N")  and  tag2.startswith("N"):
            if check_word_correct(lemma1) and check_word_correct(lemma2) and check_word_correct(word1) and check_word_correct(word2):
                bi=lemma1+" "+lemma2
                line_vocab.append(bi)
                last_added=True
            elif check_word_correct(lemma1) and check_word_correct(word1):
                line_vocab.append(lemma1)
                last_added=False
            else:
                last_added=False
                
            if bigram_counter==len(train_words)-2:
                if check_word_correct(lemma2) and check_word_correct(word2):
                    line_vocab.append(lemma2)
        else:
            ##unigram
            if not last_added:
                if check_word_correct(lemma1) and check_word_correct(word1):
                    line_vocab.append(lemma1)
            last_added=False
            if bigram_counter==len(train_words)-2:
                if check_word_correct(lemma2) and check_word_correct(word2):
                    line_vocab.append(lemma2)
    
    result_dict=dict()
    for voc in line_vocab:
        if word_length_filtering(voc):
            line_vocab.remove(voc)
    for key in vocab_per_post_type.keys():
        result_dict[key]=log10(post_type_len[key]/train_records)
        
    for key in vocab_per_post_type.keys():
        for voc in line_vocab:
            voc=voc.lower()
            if voc in vocab_per_post_type[key]:
                prob=((vocab_per_post_type[key][voc]+delta)/(vocab_sizes[key]+(vocabulary_len)))
                result_dict[key]=result_dict[key]+log10(prob)
            #else:
             #   voc_arr=voc.split(" ")
              #  for loop_arr in voc_arr:
               #     if check_word_correct(loop_arr):
                #        prob=(0.5/(vocab_sizes[key]+(delta*vocabulary_len)))
                 #       result_dict[key]=result_dict[key]+log10(prob)
    
    ans=max(result_dict, key=result_dict.get)
    output_one_line['pred']=ans
    output_one_line['story']=result_dict['story']
    output_one_line['show_hn']=result_dict['show_hn']
    output_one_line['ask_hn']=result_dict['ask_hn']
    output_one_line['poll']=result_dict['poll']
    output_one_line['ans']="right" if (loop_row["Post Type"]==ans) else "wrong"
    output_pred.append(ans) 
    output_lines.append(output_one_line)
    counter=counter+1
##  
import numpy as np
from sklearn.metrics import classification_report
print(classification_report(output_given, output_pred))
from sklearn.metrics import confusion_matrix
varFroCM=confusion_matrix(output_given, output_pred)
print("Confusion Matrix")
print(varFroCM)
varFroCM= varFroCM.astype('float') / varFroCM.sum(axis=1)[:, np.newaxis]
print("Per class wise accuracy:")
print(varFroCM.diagonal())
print("Overall Accuracy:",accuracy_score(output_given,output_pred)*100)

              precision    recall  f1-score   support

      ask_hn       0.81      1.00      0.89      5454
        poll       1.00      0.17      0.29         6
     show_hn       0.92      1.00      0.96      4903
       story       1.00      0.99      0.99    126852

    accuracy                           0.99    137215
   macro avg       0.93      0.79      0.78    137215
weighted avg       0.99      0.99      0.99    137215

Confusion Matrix
[[  5453      0      0      1]
 [     0      1      1      4]
 [     0      0   4903      0]
 [  1319      0    435 125098]]
Per class wise accuracy:
[0.99981665 0.16666667 1.         0.98617286]
Overall Accuracy: 98.71734139853514


In [14]:
output_file= open(result_file,"w+", encoding="utf-8")
for output_one_line in output_lines:
    output_file.write("%d  %s  %s  %.10f  %.10f  %.10f  %.10f  %s  %s\n\r" 
                      %(output_one_line['counter'], output_one_line['title'], 
                        output_one_line['pred'],output_one_line['story'],output_one_line['ask_hn'],
                        output_one_line['show_hn'],output_one_line['poll'],
                       output_one_line['trueval'],output_one_line['ans']))
    counter=counter+1
output_file.close()